In [2]:
import os
import numpy as np
import pandas as pd
import shutil

In [3]:
root_folder = './dataset/original_data/'
target_rgb = './dataset/original_data/data/rgb'
target_depth = './dataset/original_data/data/depth'

for folder in os.listdir(root_folder):
    
    if folder == 'data':
        continue
    
    print (folder)
    
    folder_dir = os.path.join(root_folder, folder)
    rgb_dir = os.path.join(folder_dir, 'RGB_black_background_dataset')
    depth_dir = os.path.join(folder_dir, 'Depth_black_background_dataset')
    
    for jj in range(len(os.listdir(rgb_dir))):
        
        original_rgb = rgb_dir + '/picture_number_' + str(jj) + '.png'
        original_depth = depth_dir + '/picture_number_' + str(jj) + '.png'
        
        new_rgb_img = f"set{folder[-1:]}_img{jj}.png"
        new_depth_img = f"set{folder[-1:]}_img{jj}.png"
        
        target_rgb_path = os.path.join(target_rgb, new_rgb_img)
        target_depth_path = os.path.join(target_depth, new_depth_img)
        
        shutil.copy(original_rgb, target_rgb_path)
        shutil.copy(original_depth, target_depth_path)


PIH_0
PIH_1
PIH_2
PIH_3
PIH_4
PIH_5
PIH_6
PIH_7
PIH_8
PIH_9


In [4]:
root_folder = './dataset/original_data/'
csv_output_path = './dataset/original_data/data/data.csv'

dataframes = []

for folder in os.listdir(root_folder):
    if folder == 'data':
        continue
    
    print (folder)
    
    folder_dir = os.path.join(root_folder, folder)
    csv_path = os.path.join(folder_dir, 'black_background_dataset.csv')
    
    if os.path.exists(csv_path):
        
        print(f"Reading data from: {csv_path}")
        
        data = pd.read_csv(csv_path)
        
        print("Sample data before modification:")
        print(data.head())
        
        data['Picture name'] = data['Picture name'].apply(lambda x: f"set{folder[-1:]}_img{x.split('_')[-1]}")
      
        print("Sample data after modification:")
        print(data.head())
        
        dataframes.append(data)
        
        # Concatenate all dataframes into a single dataframe
combined_data_df = pd.concat(dataframes, ignore_index=True)

combined_data_df.to_csv(csv_output_path, index=False)
print(f"Combined data saved to: {csv_output_path}")

PIH_0
Reading data from: ./dataset/original_data/PIH_0\black_background_dataset.csv
Sample data before modification:
   Angel [rad]  Radial distance [m]                               Picture name
0     0.020362             0.000764  black_background_dataset_picture_number_0
1     0.021588             0.005121  black_background_dataset_picture_number_1
2     0.058392             0.003897  black_background_dataset_picture_number_2
3     0.063607             0.002824  black_background_dataset_picture_number_3
4     0.083804             0.005398  black_background_dataset_picture_number_4
Sample data after modification:
   Angel [rad]  Radial distance [m] Picture name
0     0.020362             0.000764       set0_0
1     0.021588             0.005121       set0_1
2     0.058392             0.003897       set0_2
3     0.063607             0.002824       set0_3
4     0.083804             0.005398       set0_4
PIH_1
Reading data from: ./dataset/original_data/PIH_1\black_background_dataset.csv

In [5]:
# Convert angle from radians to degrees
combined_data_df['Angle [deg]'] = combined_data_df['Angel [rad]'] * (180 / np.pi)

# Convert radial distance from meters to millimeters
combined_data_df['Radial distance [mm]'] = combined_data_df['Radial distance [m]'] * 1000

# Drop the original columns
combined_data_df.drop(columns=['Angel [rad]', 'Radial distance [m]'], inplace=True)

# Reorder the columns if needed
combined_data_df = combined_data_df[['Angle [deg]', 'Radial distance [mm]', 'Picture name']]

# Write the modified data to a new CSV file
combined_data_df.to_csv(csv_output_path, index=False)

Change image names to rad and angle

In [51]:
root_folder = './dataset/data/'
csv_output_path = './dataset/data/data.csv'
rgb_dir = './dataset/data/rgb/'
depth_dir = './dataset/data/depth/'
rgb_rad_dir = './dataset/data/rgb_rad/'
rgb_angle_dir = './dataset/data/rgb_angle/'
depth_rad_dir = './dataset/data/depth_rad/'
depth_angle_dir = './dataset/data/depth_angle/'

data = pd.read_csv(csv_output_path)

for ii in range(len(data['Picture name'])):
    
    original_rgb = os.path.join(rgb_dir, data['Picture name'][ii] + '.png')
    original_depth = os.path.join(depth_dir, data['Picture name'][ii]+ '.png')
    
    rgb_rad_name = str(data['Radial distance [mm]'][ii]).replace('.', '_') + '.png'
    rgb_angle_name = str(data['Angle [deg]'][ii]).replace('.', '_') + '.png'
    depth_rad_name = str(data['Radial distance [mm]'][ii]).replace('.', '_') + '.png'
    depth_angle_name = str(data['Angle [deg]'][ii]).replace('.', '_') + '.png'
    
    target_rgb_rad = os.path.join(rgb_rad_dir, rgb_rad_name)
    target_rgb_angle = os.path.join(rgb_angle_dir, rgb_angle_name)
    target_depth_rad = os.path.join(depth_rad_dir, depth_rad_name)
    target_depth_angle = os.path.join(depth_angle_dir, depth_angle_name)

    shutil.copy(original_rgb, target_rgb_rad)
    shutil.copy(original_rgb, target_rgb_angle)
    shutil.copy(original_depth, target_depth_rad)
    shutil.copy(original_depth, target_depth_angle)

centering

In [56]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to center the blue object in both RGB and depth images
def center_blue_object(rgb_image, depth_image):
    # Convert the RGB image to the HSV color space
    hsv = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds of the blue color in HSV
    lower_blue = np.array([90, 100, 100])
    upper_blue = np.array([130, 255, 255])

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Find contours of the blue object
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the largest contour (assuming it's the blue object)
        max_contour = max(contours, key=cv2.contourArea)
        
        # Calculate the centroid of the contour
        M = cv2.moments(max_contour)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])

        # Calculate the offset to move the centroid to the center of the image
        rows, cols = rgb_image.shape[:2]
        offset_x = cols // 2 - cx
        offset_y = rows // 2 - cy

        # Shift the RGB image to center the blue object
        centered_rgb_image = np.roll(rgb_image, offset_x, axis=1)
        centered_rgb_image = np.roll(centered_rgb_image, offset_y, axis=0)

        # Shift the depth image using the same offset
        centered_depth_image = np.roll(depth_image, offset_x, axis=1)
        centered_depth_image = np.roll(centered_depth_image, offset_y, axis=0)

        return centered_rgb_image, centered_depth_image
    else:
        # No blue object detected, return the original images
        return rgb_image, depth_image



In [58]:
# Path to the folder containing the original images
original_rgb_folder = './dataset/original_data/rgb/'
original_depth_folder = './dataset/original_data/depth/'

# Path to the folder to save the centered images
centered_rgb_folder = './dataset/centered_data/rgb/'
centered_depth_folder = './dataset/centered_data/depth/'

set_number = '2'

for image_name in os.listdir(original_rgb_folder):
    if image_name.startswith(f'set{set_number}'):
        # Load the RGB and depth images
        rgb_image_path = os.path.join(original_rgb_folder, image_name)
        depth_image_path = os.path.join(original_depth_folder, image_name)
        rgb_image = cv2.imread(rgb_image_path)
        depth_image = cv2.imread(depth_image_path)

        # Center the blue object in both RGB and depth images
        centered_rgb_image, centered_depth_image = center_blue_object(rgb_image, depth_image)

        # Save the centered images
        centered_rgb_path = os.path.join(centered_rgb_folder, image_name)
        centered_depth_path = os.path.join(centered_depth_folder, image_name)
        cv2.imwrite(centered_rgb_path, centered_rgb_image)
        cv2.imwrite(centered_depth_path, centered_depth_image)
